# Neural Network Testing with less noise Data

### Get Libraries and Functions

In [1]:
import sys 
sys.path.append("../")

import torch 
import torch.nn

from Conv1d_NN import Conv1d_NN

from models import * 
from train import * 
from dataset import * 



### Less Noise Data

In [2]:
less_noise_dataset = MNIST1D_Dataset()
less_noise_dataset.data_args.corr_noise_scale = 0.1
less_noise_dataset.data_args.iid_noise_scale = 0.01

less_noise_data = less_noise_dataset.make_dataset()


### Regular Conv1d_NN

In [3]:
# Regular Conv1d_NN
model1 = nn.Sequential(
    Conv1d_NN(in_channels=1, out_channels =32, K = 3),  
    nn.ReLU(), 
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    Conv1d_NN(in_channels=32, out_channels=32, K = 3),
    nn.ReLU(),
    nn.Flatten(), 
    nn.Linear(1280, 10)
)

# Get summary 
from torchsummary import summary
summary(model1, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 40]           3,104
         Conv1d_NN-5               [-1, 32, 40]               0
              ReLU-6               [-1, 32, 40]               0
            Conv1d-7               [-1, 32, 40]           3,104
         Conv1d_NN-8               [-1, 32, 40]               0
              ReLU-9               [-1, 32, 40]               0
          Flatten-10                 [-1, 1280]               0
           Linear-11                   [-1, 10]          12,810
Total params: 19,146
Trainable params: 19,146
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [4]:
# Regular Conv1d_NN Results 
print("Regular Conv1d_NN Results")
results1 = train_model(less_noise_data, model1, less_noise_dataset.model_args)

Regular Conv1d_NN Results
step 1000, dt 21.36s, train_loss 6.033e-01, test_loss 1.060e+00, train_acc 73.5, test_acc 60.4
step 2000, dt 17.17s, train_loss 4.478e-01, test_loss 1.302e+00, train_acc 80.8, test_acc 62.8
step 3000, dt 15.10s, train_loss 4.920e-01, test_loss 1.312e+00, train_acc 79.8, test_acc 59.5
step 4000, dt 4018.07s, train_loss 3.084e-01, test_loss 1.618e+00, train_acc 83.9, test_acc 58.6
step 5000, dt 764.86s, train_loss 3.204e-01, test_loss 1.618e+00, train_acc 85.9, test_acc 61.2
step 6000, dt 15.29s, train_loss 2.920e-01, test_loss 1.787e+00, train_acc 85.0, test_acc 61.0


### Local -> Global Neural Network

In [5]:
# Local -> Global Conv1d_NN
model2 = nn.Sequential(
   nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model2, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 38]             128
              ReLU-2               [-1, 32, 38]               0
            Conv1d-3               [-1, 32, 38]           3,104
         Conv1d_NN-4               [-1, 32, 38]               0
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 36]           3,104
              ReLU-7               [-1, 32, 36]               0
            Conv1d-8               [-1, 32, 36]           3,104
         Conv1d_NN-9               [-1, 32, 36]               0
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 34]           3,104
             ReLU-12               [-1, 32, 34]               0
           Conv1d-13               [-1, 32, 34]           3,104
        Conv1d_NN-14               [-1,

In [6]:
# Local -> Global Conv1d_NN Results 
print("Local -> Global Conv1d_NN Results")
results2 = train_model(less_noise_data, model2, less_noise_dataset.model_args)

Local -> Global Conv1d_NN Results
step 1000, dt 127.69s, train_loss 1.631e-01, test_loss 2.645e-01, train_acc 94.9, test_acc 92.5
step 2000, dt 67.66s, train_loss 1.690e-01, test_loss 2.593e-01, train_acc 93.5, test_acc 92.9
step 3000, dt 87.92s, train_loss 1.440e-01, test_loss 1.567e-01, train_acc 96.0, test_acc 94.9
step 4000, dt 62.93s, train_loss 1.582e-01, test_loss 1.299e-01, train_acc 96.0, test_acc 96.3
step 5000, dt 309.92s, train_loss 1.870e-01, test_loss 1.240e-01, train_acc 97.2, test_acc 96.7
step 6000, dt 17.84s, train_loss 9.656e-02, test_loss 1.519e-01, train_acc 95.7, test_acc 95.2


### Global -> Local Neural Network

In [7]:
# Global -> Local Conv1d_NN
model3 = nn.Sequential(
   Conv1d_NN(in_channels=1, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(),
   Conv1d_NN(in_channels=32, out_channels=32, K = 3),
   nn.ReLU(),
   nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3),
   nn.ReLU(), 
   nn.Flatten(),
   nn.Linear(1088, 10)
)

# Get summary 
from torchsummary import summary 
summary(model3, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 40]             128
         Conv1d_NN-2               [-1, 32, 40]               0
              ReLU-3               [-1, 32, 40]               0
            Conv1d-4               [-1, 32, 38]           3,104
              ReLU-5               [-1, 32, 38]               0
            Conv1d-6               [-1, 32, 38]           3,104
         Conv1d_NN-7               [-1, 32, 38]               0
              ReLU-8               [-1, 32, 38]               0
            Conv1d-9               [-1, 32, 36]           3,104
             ReLU-10               [-1, 32, 36]               0
           Conv1d-11               [-1, 32, 36]           3,104
        Conv1d_NN-12               [-1, 32, 36]               0
             ReLU-13               [-1, 32, 36]               0
           Conv1d-14               [-1,

In [8]:
# Global -> Local Conv1d_NN Results 
print("Global -> Local Conv1d_NN Results")
results3 = train_model(less_noise_data, model3, less_noise_dataset.model_args)

Global -> Local Conv1d_NN Results
step 1000, dt 17.15s, train_loss 3.721e-01, test_loss 3.213e-01, train_acc 91.2, test_acc 88.9
step 2000, dt 16.75s, train_loss 1.474e-01, test_loss 2.357e-01, train_acc 96.2, test_acc 92.5
step 3000, dt 17.16s, train_loss 7.028e-02, test_loss 1.630e-01, train_acc 97.0, test_acc 94.6
step 4000, dt 16.90s, train_loss 3.292e-02, test_loss 1.986e-01, train_acc 96.5, test_acc 94.0
step 5000, dt 19.61s, train_loss 4.868e-02, test_loss 2.276e-01, train_acc 97.1, test_acc 94.6
step 6000, dt 18.29s, train_loss 3.339e-02, test_loss 2.569e-01, train_acc 96.0, test_acc 93.1


### Branching Neural Network

In [9]:
model4 = nn.Sequential(
   BranchingNetwork(in_ch = 1, out_ch1 = 16, out_ch2=16, kernel_size = 3), 
   BranchingNetwork(in_ch = 16, out_ch1 = 8, out_ch2=8, kernel_size = 3),
   BranchingNetwork(in_ch = 8, out_ch1 = 4, out_ch2=4, kernel_size = 3),
   nn.Flatten(), 
   nn.Linear(160, 10)
)


# Get summary
from torchsummary import summary 
summary(model4, (1, 40), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 38]              64
              ReLU-2               [-1, 16, 38]               0
            Conv1d-3               [-1, 16, 40]              64
         Conv1d_NN-4               [-1, 16, 40]               0
              ReLU-5               [-1, 16, 40]               0
            Conv1d-6               [-1, 16, 40]             528
  BranchingNetwork-7               [-1, 16, 40]               0
            Conv1d-8                [-1, 8, 38]             392
              ReLU-9                [-1, 8, 38]               0
           Conv1d-10                [-1, 8, 40]             392
        Conv1d_NN-11                [-1, 8, 40]               0
             ReLU-12                [-1, 8, 40]               0
           Conv1d-13                [-1, 8, 40]             136
 BranchingNetwork-14                [-1

In [10]:
# Branching Network Results 
print("Branching Conv1d_NN Conv1d Results")
results4 = train_model(less_noise_data, model4, less_noise_dataset.model_args)

Branching Conv1d_NN Conv1d Results
step 1000, dt 26.54s, train_loss 4.663e-02, test_loss 1.503e-01, train_acc 97.5, test_acc 95.5
step 2000, dt 23.58s, train_loss 1.736e-02, test_loss 1.099e-01, train_acc 98.2, test_acc 97.2
step 3000, dt 23.40s, train_loss 5.772e-03, test_loss 1.051e-01, train_acc 98.6, test_acc 97.0
step 4000, dt 22.93s, train_loss 2.300e-03, test_loss 1.004e-01, train_acc 99.5, test_acc 98.3
step 5000, dt 23.97s, train_loss 9.242e-03, test_loss 7.192e-02, train_acc 99.5, test_acc 98.5
step 6000, dt 23.51s, train_loss 1.548e-01, test_loss 1.210e-01, train_acc 98.5, test_acc 96.2
